In [25]:
import psycopg2
import pandas as pd
import numpy as np

In [27]:
# NEVER share passwords

dbname = "dev"
host = ""
port = 5439
user = ""
password = ""

In [28]:
# connect to redshift

connect = psycopg2.connect(
        dbname=dbname, host=host, port=port, user=user, password=password
    )

cursor = connect.cursor()

In [29]:
# query all rows and columns from online_transactions table

query = """select *
           from bootcamp1.stock_description
           limit 10
           """

pd.read_sql(query, connect)

,stock_code,description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE
5,10124G,ARMY CAMO BOOKCOVER TAPE
6,10125,MINI FUNKY DESIGN TAPES
7,10133,COLOURING PENCILS BROWN TUBE
8,10135,COLOURING PENCILS BROWN TUBE
9,11001,ASSTD DESIGN RACING CAR PEN


In [30]:
# query all rows and columns from online_transactions table

query = """select *
           from bootcamp1.online_transactions
           """

online_transactions = pd.read_sql(query, connect)
online_transactions

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
0,536365,84406B,8,2010-12-01 08:26:00,2.75,u1785,United Kingdom
1,536365,21730,6,2010-12-01 08:26:00,4.25,u1785,United Kingdom
2,536368,22913,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom
3,536367,22745,6,2010-12-01 08:34:00,2.10,u13047,United Kingdom
4,536367,84969,6,2010-12-01 08:34:00,4.25,u13047,United Kingdom
...,...,...,...,...,...,...,...
541905,581586,23275,24,2011-12-09 12:49:00,1.25,u13113,United Kingdom
541906,581587,22556,12,2011-12-09 12:50:00,1.65,u1268,France
541907,581587,22726,4,2011-12-09 12:50:00,3.75,u1268,France
541908,581587,23256,4,2011-12-09 12:50:00,4.15,u1268,France


In [31]:
# query all rows and columns from stock_description table

query = """select *
           from bootcamp1.stock_description
           """

stock_description = pd.read_sql(query, connect)
stock_description


,stock_code,description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE
...,...,...
3947,DCGSSGIRL,GIRLS PARTY BAG
3948,DOT,DOTCOM POSTAGE
3949,PADS,PADS TO MATCH ALL CUSHIONS
3950,POST,POSTAGE


In [32]:
type(online_transactions)

pandas.core.frame.DataFrame

In [33]:
type(stock_description)

pandas.core.frame.DataFrame

In [34]:
online_transactions.shape

# 54k rows and 7 columns

(541910, 7)

In [35]:
stock_description.shape

# 4k rows and 2 columns

(3952, 2)

In [17]:
# check for duplicated data

online_transactions.duplicated().sum()

# 5270 rows of duplicated data

5270

In [18]:
# preview the duplicated data

online_transactions[online_transactions.duplicated()]

# we can either drop all the duplicated data or keep only one

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
293,536464,21992,1,2010-12-01 12:23:00,2.95,u17968,United Kingdom
373,536528,84985A,1,2010-12-01 13:17:00,1.45,u15525,United Kingdom
456,536412,22327,1,2010-12-01 11:49:00,2.95,u1792,United Kingdom
464,536412,22273,1,2010-12-01 11:49:00,2.95,u1792,United Kingdom
469,536412,21708,1,2010-12-01 11:49:00,4.95,u1792,United Kingdom
...,...,...,...,...,...,...,...
541412,581450,22118,1,2011-12-08 17:54:00,1.25,u16794,United Kingdom
541440,581456,35964,2,2011-12-08 18:42:00,0.39,u1753,United Kingdom
541843,581538,22694,1,2011-12-09 11:34:00,2.10,u14446,United Kingdom
541846,581538,23318,1,2011-12-09 11:34:00,2.49,u14446,United Kingdom


In [37]:
# an example of duplicated data
# both rows are identical

online_transactions[(online_transactions.invoice == "536464") &
                   (online_transactions.stock_code == "21992")]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
4654,536464,21992,1,2010-12-01 12:23:00,2.95,u17968,United Kingdom
4665,536464,21992,1,2010-12-01 12:23:00,2.95,u17968,United Kingdom


In [44]:
# using describe to look at the distribution of data

online_transactions.describe()

,quantity,price
count,541910.000000,541910.000000
mean,9.552234,4.611138
std,218.080957,96.759765
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


In [45]:
# using describe to look at the distribution of data

stock_description.describe()

# hmm.. Iulia spotted there were 47 cases of ? in description, let's investigate

,stock_code,description
count,3952,3952
unique,3905,3785
top,22600,?
freq,2,47


In [46]:
# oh no... we have ?, and 6 stock codes with the same description and other duplicates appearing. This is such a mess!
# we may need to recalculate the stock code

stock_description.description.value_counts()

?                                     47
METAL SIGN,CUPCAKE SINGLE HOOK         6
CINAMMON SET OF 9 T-LIGHTS             2
COLUMBIAN CANDLE RECTANGLE             2
3 WHITE CHOC MORRIS BOXED CANDLES      2
                                      ..
MAGNETS PACK OF 4 SWALLOWS             1
MAGNETS PACK OF 4 CHILDHOOD MEMORY     1
MAGNETS PACK OF 4 HOME SWEET HOME      1
MAGNETS PACK OF 4 VINTAGE COLLAGE      1
SAMPLES                                1
Name: description, Length: 3785, dtype: int64

In [47]:
# stock codes where description is ?

stock_description[stock_description.description == "?"]

,stock_code,description
36,16020C,?
64,16207B,?
390,21145,?
451,21232,?
535,21368,?
584,21427,?
594,21446,?
675,21591,?
865,21877,?
997,22077,?


In [38]:
# invoices where price is less than 0.. oh! we see we have missing customer ids in this case

online_transactions[online_transactions.price < 0]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
298534,A563187,B,1,2011-08-12 14:52:00,-11062.06,,United Kingdom
348948,A563186,B,1,2011-08-12 14:51:00,-11062.06,,United Kingdom


In [39]:
# invoices where quantity is less than 0

online_transactions[online_transactions.quantity < 0]

# we may want to remove rows of data where quantity < 0 in future iterations

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
38,C536383,35004C,-1,2010-12-01 09:49:00,4.65,u15311,United Kingdom
59,C536391,21980,-24,2010-12-01 10:24:00,0.29,u17548,United Kingdom
115,C536379,D,-1,2010-12-01 09:41:00,27.50,u14527,United Kingdom
139,C536391,21983,-24,2010-12-01 10:24:00,0.29,u17548,United Kingdom
140,C536391,22553,-24,2010-12-01 10:24:00,1.65,u17548,United Kingdom
...,...,...,...,...,...,...,...
541453,C581465,23660,-2,2011-12-08 18:59:00,1.65,u15755,United Kingdom
541454,C581466,22838,-1,2011-12-08 19:20:00,14.95,u13883,United Kingdom
541455,C581466,21232,-1,2011-12-08 19:20:00,1.25,u13883,United Kingdom
541468,C581470,23084,-4,2011-12-08 19:28:00,2.08,u17924,United Kingdom


In [43]:
# where one invoice has a negative quantity less than 0

online_transactions.quantity.min()

-80995

In [19]:
# look for missing data in both tables

online_transactions.isnull().sum()

# whoa, what is going on here. we just saw some missing customer ids. Looks like
# when migrating the data to redshift, the missing data was not handled correctly

invoice         0
stock_code      0
quantity        0
invoice_date    0
price           0
customer_id     0
country         0
dtype: int64

In [40]:
# but I can try to fix this by replacing the blanks with python recognised nan

online_transactions = online_transactions.replace('', np.nan)                   # to get rid of empty values

# we will fix this is in our transformation tasks

In [41]:
# check for missing values again

online_transactions.isnull().sum()

# 135k missing customer ids!!!! We will drop invoices with missing customer ids in our transformation tasks

invoice              0
stock_code           0
quantity             0
invoice_date         0
price                0
customer_id     135080
country              0
dtype: int64

In [42]:
# percentage of customers without an id

round(online_transactions.customer_id.isnull().sum()/len(online_transactions.customer_id.unique()))

31

In [22]:
# look for missing data in stock description table

stock_description.isnull().sum()

# hmm...

stock_code     0
description    0
dtype: int64

In [31]:
# fix missing data

stock_description = stock_description.replace('', np.nan)   
stock_description.isnull().sum()

# ok, cant see any missing data right now, but looks like someone replaced the missing values with ?

stock_code     0
description    0
dtype: int64

In [11]:
# check for the summary of both tables

online_transactions.info()

# invoice_date is appearing as data type object and not date time, everything else looks good

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice       541910 non-null  object 
 1   stock_code    541910 non-null  object 
 2   quantity      541910 non-null  int64  
 3   invoice_date  541910 non-null  object 
 4   price         541910 non-null  float64
 5   customer_id   541910 non-null  object 
 6   country       541910 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 28.9+ MB


In [12]:
stock_description.info()

# loos good, but will double check how much stock codes are in the online_transactions table

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3952 entries, 0 to 3951
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   stock_code   3952 non-null   object
 1   description  3952 non-null   object
dtypes: object(2)
memory usage: 61.9+ KB


In [13]:
len(online_transactions.stock_code.unique())

# hmm.. why does online_transactions have more stock codes?

4070

In [17]:
# writing a query to identify how many stock codes have been purchased but are not in the stock description database

query = """select count(distinct t1.stock_code)
           from bootcamp1.online_transactions t1
           left join bootcamp1.stock_description t2 on t1.stock_code = t2.stock_code
           where t2.stock_code is null
           """

pd.read_sql(query, connect)



,count
0,165


In [20]:
# looking at the data where stock codes are missing in the stock description table

query = """select *
           from bootcamp1.online_transactions t1
           left join bootcamp1.stock_description t2 on t1.stock_code = t2.stock_code
           where t2.stock_code is null
           """

pd.read_sql(query, connect)

# one observation, there is a missing customer id so will filter out the missing customer_id

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,stock_code,description
0,536408,21705,12,2010-12-01 11:41:00,1.65,u14307,United Kingdom,None,None
1,536500,46000M,10,2010-12-01 12:35:00,1.55,u17377,United Kingdom,None,None
2,536545,21134,1,2010-12-01 14:32:00,0.00,,United Kingdom,None,None
3,536544,21703,1,2010-12-01 14:32:00,0.85,,United Kingdom,None,None
4,536552,20950,1,2010-12-01 14:34:00,0.00,,United Kingdom,None,None
...,...,...,...,...,...,...,...,...,...
2374,581221,23444,1,2011-12-08 09:40:00,15.00,u17856,United Kingdom,None,None
2375,581266,21703,12,2011-12-08 11:25:00,0.42,u12621,Germany,None,None
2376,581439,21704,3,2011-12-08 16:30:00,1.63,,United Kingdom,None,None
2377,581469,21704,1,2011-12-08 19:28:00,0.85,u14606,United Kingdom,None,None


In [35]:
# looking at the data where stock codes are missing in the stock description table

query = """select count(distinct t1.stock_code)
           from bootcamp1.online_transactions t1
           left join bootcamp1.stock_description t2 on t1.stock_code = t2.stock_code
           where t2.stock_code is null
            and customer_id <> ''
           """

pd.read_sql(query, connect)

# one observation, there is a missing customer id so will filter out the missing customer_id
# only 22 stock codes 

,count
0,22


In [36]:
# looking at the data where stock codes are missing in the stock description table

query = """select distinct t1.stock_code
           from bootcamp1.online_transactions t1
           left join bootcamp1.stock_description t2 on t1.stock_code = t2.stock_code
           where t2.stock_code is null
            and customer_id <> ''
           """

pd.read_sql(query, connect)

# one observation, there is a missing customer id so will filter out the missing customer_id
# only 22 stock codes 

,stock_code
0,46000S
1,21705
2,21704
3,D
4,22889
5,16151A
6,84247K
7,CRUK
8,23702
9,46000M


In [7]:
connect.close()

Data Quality Audit
- We have 5270 rows of duplicated data and we keep one instance
- 31% of customers did not have customer id
- We need to fix the invoice_date field
- We need to do further investigation on stock code, but 22 stock codes without customer_id do not have a descriptions.
- We have 10k invoices with a negative quantity, with one invoice having a negative quantity of 81k. Thanks Faith!
- We have spotted 47 examples of the description having a ?, which we will replace with Unknown. Thanks Iulia!
